In [1]:
import keras
import tensorflow
import h5py
import numpy as np
import chowdhury_s_model_builder
import os

import tensorflow as tf
from keras.api.models import Sequential, Model
from keras.api.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, LSTM, Input, GlobalAveragePooling2D
from keras.api.optimizers import Adam
from keras import regularizers

2025-04-05 15:31:20.805927: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-05 15:31:20.815316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743838280.825947    8310 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743838280.830953    8310 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743838280.840028    8310 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Open the HDF5 file in read mode
with h5py.File("Data/Mid-Level_Perceptual_Features_Data.h5", "r") as hf:
    mid_level_train_data = hf["train"][:]  # Load the train dataset
    mid_level_label_data = hf["label"][:]  # Load the label dataset

print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
print(mid_level_label_data.shape, mid_level_label_data.dtype)

(5000, 256, 1292) float32
(5000, 7) int8


In [3]:
# Open the HDF5 file in read mode
with h5py.File("Data/Emotify_Data.h5", "r") as hf:
    emotify_train_data = hf["train"][:]  # Load the train dataset
    emotify_label_data = hf["label"][:]  # Load the label dataset

print(emotify_train_data.shape, emotify_train_data.dtype)  # Check the shape and type
print(emotify_label_data.shape, emotify_label_data.dtype)

(400, 256, 1292) float32
(400, 9) int8


In [4]:
mid_level_train_data = mid_level_train_data[..., np.newaxis]

print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
print(mid_level_label_data.shape, mid_level_label_data.dtype)

(5000, 256, 1292, 1) float32
(5000, 7) int8


In [5]:
emotify_train_data = emotify_train_data[..., np.newaxis]

print(emotify_train_data.shape, emotify_train_data.dtype)  # Check the shape and type
print(emotify_label_data.shape, emotify_label_data.dtype)

(400, 256, 1292, 1) float32
(400, 9) int8


In [6]:
mid_level_input_shape = mid_level_train_data.shape[1], mid_level_train_data.shape[2], mid_level_train_data.shape[3]
emotify_input_shape = emotify_train_data.shape[1], emotify_train_data.shape[2], emotify_train_data.shape[3]
print(mid_level_input_shape, emotify_input_shape)

(256, 1292, 1) (256, 1292, 1)


In [7]:
with tensorflow.device('/GPU:0'):
    input = Input(shape=(256, 1292, 1))
    x =  Conv2D(64, (5, 5), strides=2, activation="relu", padding="valid")(input)
    x =  BatchNormalization()(x)   
    # 2nd Layer
    x =  Conv2D(64, (3, 3), strides=1, activation="relu", padding="same")(x)
    x =  BatchNormalization()(x)    
    # 3rd Layer
    x =  MaxPooling2D((2, 2))(x)
    x =  Dropout(0.3)(x)    
    # 4th Layer
    x =  Conv2D(128, (3, 3), strides=1, activation="relu", padding="same")(x)
    x =  BatchNormalization()(x)    
    # 5th Layer
    x =  Conv2D(128, (3, 3), strides=1, activation="relu", padding="same")(x)
    x =  BatchNormalization()(x)    
    # 6th Layer
    x =  MaxPooling2D((2, 2))(x)
    x =  Dropout(0.3)(x)    
    # 7th Layer
    x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
    x =  BatchNormalization()(x)    
    # 8th Layer
    x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
    x =  BatchNormalization()(x)    
    # 9th Layer
    x =  Conv2D(384, (3, 3), strides=1, activation="relu", padding="same")(x)
    x =  BatchNormalization()(x)    
    # 10th Layer
    x =  Conv2D(512, (3, 3), strides=1, activation="relu", padding="same")(x)
    x =  BatchNormalization()(x)    
    # 11th Layer
    x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
    x =  BatchNormalization()(x) 
    # 12th Layer
    # x = tfa.layers.AdaptiveAveragePooling2D(x)
    x =  GlobalAveragePooling2D(keepdims=True)(x)
    x =  Flatten()(x)
    x = keras.layers.Dense(256)(x)
    A2Mid2E_branch = keras.layers.Dense(7, activation="sigmoid")(x)
    model = Model(inputs=input, outputs=A2Mid2E_branch, name="Mid-Level_Features")
    model.summary()

I0000 00:00:1743838305.428893    8310 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1743838305.434128    8310 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "Mid-Level_Features"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 1292, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 126, 644, 64)   │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 644, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 126, 644, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 126, 644, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 322, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 322, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 63, 322, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 63, 322, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 63, 322, 128)   │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 63, 322, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 31, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 31, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 31, 161, 256)   │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 31, 161, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 31, 161, 256)   │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 31, 161, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 31, 161, 384)   │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 31, 161, 384)   │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 31, 161, 512)   │     1,769,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 31, 161, 512)   │         2,048 │
│ (BatchNormalization)            │                        │             

 Total params: 5,056,071 (19.29 MB)

 Trainable params: 5,051,975 (19.27 MB)

 Non-trainable params: 4,096 (16.00 KB)

In [8]:
model.compile(optimizer=Adam(learning_rate=0.0005),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )

In [9]:
model.fit(mid_level_train_data, mid_level_label_data,
                       batch_size=8,
                       epochs=10,
                       shuffle=True, verbose=1)

Epoch 1/10


I0000 00:00:1743838500.641326    8404 service.cc:152] XLA service 0x7f5e7c009990 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743838500.643016    8404 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-04-05 15:35:00.893952: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743838501.808843    8404 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-05 15:35:03.237472: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2254', 8 bytes spill stores, 8 bytes spill loads

2025-04-05 15:35:03.396843: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/nigelchua0412/.local/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/nigelchua0412/.local/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/nigelchua0412/.local/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/nigelchua0412/.local/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/nigelchua0412/.local/lib/python3.12/site-packages/ipykernel/kernelbas